In [1]:
import pandas as pd
import re, nltk, random, os, json
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

In [2]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 'distilbert-base-uncased-distilled-squad', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'use_cuda': False})

In [3]:
idx = 0
to_predict = []
for context in context_sentences:
    qas = []
    for prop in cand_properties:
        qas.append({'question': cand_properties[prop], 'id': idx})
        idx += 1
    to_predict.append({'context': context, 'qas':qas})

NameError: name 'context_sentences' is not defined

In [116]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.,]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc,enableStopword=False):
#     doc = regex.sub(' ', doc)
    if enableStopword:
        doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower().replace("\n","")

df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")
for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
    df_wiki[col] = [preprocessingText(str(i), False) for i in df_wiki[col]]
    
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [64]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/jarana/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
df_wiki_in_domains = df_wiki[df_wiki.overview.str.contains("|".join(domains))]

NameError: name 'domains' is not defined

In [266]:
import glob, re, math

def camel_case_split(str): 
    words = [[str[0]]] 
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 

    return " ".join([i.lower() for i in [''.join(word) for word in words]])    

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

schema = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/schema.json")
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/dev/schema.json")])
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/test/schema.json")])

tmp = []
special = {"buse": "bus", "rentalcar": "rent car", "ridesharing": "taxi", "messaging": "message"}
for i in schema.service_name.tolist():
    n = i.split("_")[0].lower()
    if n[-1] == "s":
        n = n[:-1]
    if n in special:
        n = special[n]
    
    tmp.append(n)
schema['service_name'] = tmp
domains = schema.service_name.unique().tolist()

orgType = pd.read_csv("data/dialog/all-layers-types.csv")
orgProp = pd.read_csv("data/dialog/all-layers-properties.csv")

schemaCorpus = [camel_case_split(i[0]) + " #SEP# " + cleanhtml(i[1]).lower() for i in orgType[['label', 'comment']].values]
bm25s = BM25L([doc.split() for doc in schemaCorpus])

sProp2Desc = {camel_case_split(i[0]):cleanhtml(i[1]).lower() for i in np.concatenate([orgProp[['label', 'comment']].values, orgType[['label', 'comment']].values])}
sType2Prop = {}
sType2Subtype = {}
for i,j,k in orgType[['label', 'subTypeOf', 'properties']].values:
    if type(j) == str:
        sType2Subtype[camel_case_split(i)] = [camel_case_split(z) for z in j.replace("http://schema.org/", "").split(", ")]
                           
    if type(k) == str:
        sType2Prop[camel_case_split(i)] = [camel_case_split(z) for z in k.replace("http://schema.org/", "").split(", ")]
    

In [223]:
corpus = []
doc2title = {}
for name, group in df_wiki.groupby("title"):
    doc = []
    for col in ["overview", "sectionLabel", "headline", "text"]:
#     for col in ["title", "text"]:
        doc.append(" ".join(group[col].unique().tolist()))
        
    doc = name+" #SEP# "+ " ".join(doc)
    corpus.append(doc)
    doc2title[doc] = name
# corpus = df_wiki.overview.unique().tolist()
# bm25 = BM25Plus([doc.split() for doc in corpus])
bm25w = BM25L([doc.split() for doc in corpus])
# bm25 = BM25Okapi([tokenizer.tokenize(doc) for doc in corpus])

In [216]:
 bm25s.get_top_n("flight".split(), schemaCorpus, n=10)

['reservation hold#SEP#the status of a reservation on hold pending an update like credit card number or flight changes.',
 'flight reservation#SEP#a reservation for air travel.note: this type is for information about actual reservations, e.g. in confirmation emails or html pages with individual confirmations of reservations. for offers of tickets, use offer.',
 'check out action#SEP#the act of an agent communicating (service provider, social media, etc) their departure of a previously reserved service (e.g. flight check in) or place (e.g. hotel).related actions:checkinaction: the antonym of checkoutaction.departaction: unlike departaction, checkoutaction implies that the agent is informing/confirming the end of a previously reserved service.cancelaction: unlike cancelaction, checkoutaction implies that the agent is informing/confirming the end of a previously reserved service.',
 'check in action#SEP#the act of an agent communicating (service provider, social media, etc) their arrival 

In [ ]:
from textblob import TextBlob
import nltk
unknown_services = ['alarm', 'weather']
matches = []
for service, desc in schema[['service_name', 'description']].values:
#   ignore weather and alarm
    if service in unknown_services:
        continue
        
    types = [i.split(" #SEP# ")[0] for i in bm25s.get_top_n(service.split(), schemaCorpus, n=10)]
    wiki2desc = dict(i.split(" #SEP# ") for i in bm25w.get_top_n(preprocessingText(desc).split(), corpus, n=10))
    
#   get subtypes
    subtypes = []
    for t in types:
        if t in sType2Subtype:
            subtypes.extend(sType2Subtype[t])
    subtypes = list(set(subtypes))
    
    cand_props = []
    for t in types:
        if t in sType2Prop:
            cand_props.extend(sType2Prop[t])
    cand_props += types
    
    seen_prop = []
    for prop in cand_props:
        if prop in schema.service_name.unique().tolist():
            continue
        propLast = prop.split()[-1]
        for wiki in wiki2desc:
            for sentence in wiki2desc[wiki].split(". "):
                found_exact_match = False
                exact_match_start = sentence.find(prop)
                if exact_match_start > -1:
                    matches.append([sProp2Desc[prop], prop, exact_match_start, sentence])
                    found_exact_match = True
                    
                if propLast not in seen_prop and not found_exact_match:
                    exact_match_start = sentence.find(propLast)
                    if exact_match_start > -1:
                        matches.append([sProp2Desc[raw_prop], propLast, exact_match_start, sentence])
        seen_prop.append(propLast)
        
        
#     TODO split nouns
#     tokenized = nltk.word_tokenize(desc.lower())
#     nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')]
# print(matches)

In [270]:
matches

[['a fundingscheme combines organizational, project and policy aspects of grant-based funding    that sets guidelines, principles and mechanisms to support other kinds of projects and activities.    funding is typically organized via grant funding. examples of funding schemes: swiss priority programmes (spps); eu framework 7 (fp7); horizon 2020; the nih-r01 grant program; wellcome institutional strategic support fund. for large scale public sector funding, the management and administration of grant awards is often handled by other, dedicated, organizations - fundingagencys such as erc, rea, ...',
  'type',
  18,
  'choose an account type'],
 ['a fundingscheme combines organizational, project and policy aspects of grant-based funding    that sets guidelines, principles and mechanisms to support other kinds of projects and activities.    funding is typically organized via grant funding. examples of funding schemes: swiss priority programmes (spps); eu framework 7 (fp7); horizon 2020; the

In [263]:
sProp2Desc['address']

'physical address of the item.'

In [264]:
cand_props

['actionable feedback policy',
 ' additional property',
 ' additional type',
 ' address',
 ' aggregate rating',
 ' alternate name',
 ' alumni',
 ' amenity feature',
 ' area served',
 ' award',
 ' awards',
 ' branch code',
 ' branch of',
 ' brand',
 ' contact point',
 ' contact points',
 ' contained in',
 ' contained in place',
 ' contains place',
 ' corrections policy',
 ' currencies accepted',
 ' department',
 ' description',
 ' disambiguating description',
 ' dissolution date',
 ' diversity policy',
 ' diversity staffing report',
 ' duns',
 ' email',
 ' employee',
 ' employees',
 ' ethics policy',
 ' event',
 ' events',
 ' fax number',
 ' fees and commissions specification',
 ' founder',
 ' founders',
 ' founding date',
 ' founding location',
 ' funder',
 ' geo',
 ' geo contains',
 ' geo covered by',
 ' geo covers',
 ' geo crosses',
 ' geo disjoint',
 ' geo equals',
 ' geo intersects',
 ' geo overlaps',
 ' geo touches',
 ' geo within',
 ' global location number',
 ' has credential',


In [249]:
len(matches)

23

In [136]:
matches

[['the type of a bank account.', 'type', 18, 'choose an account type'],
 ['the type of a bank account.',
  'type',
  26,
  ' there are many different types of bank accounts available through swiss banks'],
 ['the type of a bank account.',
  'type',
  162,
  "certain bank accounts require a minimum amount for the initial deposit, a minimum account balance, and other qualifying factors.compare the requirements for each type of account you're interested in opening to find one that will work best for your financial needs.some banks require customers to use swiss francs (chf)"],
 ['the type of a bank account.',
  'type',
  317,
  "for example, bank representatives may request a copy of a contract proving that a house was sold to acquire the money in question, or receipts from securities sales, or a verifying statement from a bank you've done business with in the past.banks may verify the listed address of a potential customer by sending some type of correspondence through the mail"],
 ['the

In [145]:
set([i[1] for i in matches])

{'account',
 'airport',
 'album',
 'albums',
 'allowed',
 'amount',
 'article',
 'aspect',
 'bank',
 'bank account',
 'bus station',
 'bus stop',
 'bus trip',
 'business',
 'cancelled',
 'card',
 'code',
 'complete',
 'count',
 'date',
 'dealer',
 'deposit account',
 'due',
 'duration',
 'event',
 'flight number',
 'hold',
 'hotel room',
 'limit',
 'local business',
 'location',
 'message',
 'method',
 'music store',
 'name',
 'number',
 'object',
 'of',
 'origin',
 'page',
 'pending',
 'pickup time',
 'platform',
 'playlist',
 'presented',
 'project',
 'read',
 'recording',
 'rental',
 'role',
 'room',
 'schedule',
 'season',
 'sent',
 'service',
 'size',
 'stand',
 'station',
 'stop',
 'store',
 'taxi stand',
 'team',
 'text',
 'theater',
 'time',
 'tracks',
 'trip',
 'type',
 'used',
 'value',
 'venue',
 'wash',
 'work'}

In [535]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('bert-base-nli-cls-token')
# model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

INFO:root:Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
INFO:root:Did not find a '/' or '\' in the name. Assume to download model from server.
INFO:root:Downloading sentence transformer model from https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/roberta-base-nli-stsb-mean-tokens.zip and saving it at /Users/jarana/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
100%|██████████| 459M/459M [02:28<00:00, 3.09MB/s] 
INFO:root:Load SentenceTransformer from folder: /Users/jarana/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
INFO:root:Use pytorch device: cpu


In [539]:
# Corpus with example sentences
corpus = df_wiki[df_wiki.title == "use uber"].text.tolist()
corpus_embeddings = model.encode(corpus)

Batches: 100%|██████████| 3/3 [00:00<00:00,  3.81it/s]


In [53]:
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

In [ ]:
## queries = [camel_case_split(i) for i in list(cand_properties.keys())]
query_embeddings = model.encode(queries)

In [533]:
import scipy
import numpy as np
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist(corpus_embeddings, [query_embedding], "cosine")
    print(query)
    print(corpus[np.argmin(distances)], 1 - distances[np.argmin(distances)][0])
    print()
#     print(distances)
    


taxi stand
taxi 0.8057539461799915

pickup time
pickup 0.8259701335292039

pickup location
pickup 0.8966534259500655

party size
party, 0.7600675358242679

modified time
change 0.5604403547233994

booking time
time 0.5699787148433507

under name
name 0.760251215903371

reservation cancelled
cancel 0.6970501159768268

reservation pending
reservation, 0.7151689708786823

reservation hold
reservation, 0.7946275730114153



In [354]:
intents = []
for idx, row, in schema.iterrows():
    for i in row.intents:
        name = preprocessingText(camel_case_split(i['name']), True)
        desc = preprocessingText(camel_case_split(i['description']), True)
        intent = row.service_name + " " + row.description + " " +name + " " + desc
        intents.append(preprocessingText(intent.lower(), True))

for query in intents:
    tokenized_query = query.split()
    # tokenized_query = tokenizer.tokenize(query)
    doc = bm25.get_top_n(tokenized_query, corpus, n=10)
    print(query)
    print(doc2title[doc[0]])
    text = " ".join(df_wiki[df_wiki.title.str.contains(doc2title[doc[0]])].text.tolist())  
    terms = set(text.split())
    cand_prop = []
    for p in props:
        if p in terms:
            cand_prop.append(p)
    print(cand_prop)
    print()


bank manage bank accounts transfer money check balance check amount money user's bank account
transfer money between bank accounts
['address', 'amount', 'branch', 'code', 'email', 'identifier', 'member', 'name', 'option', 'recipient']

bank manage bank accounts transfer money transfer money transfer money one bank account another user's account
transfer money one bank another
['address', 'amount', 'branch', 'code', 'identifier', 'location', 'name', 'photo', 'procedure', 'recipient', 'review', 'steps']

bus book bus journeys biggest bus network country find bus find bus journey given pair cities
ride bus
['area', 'free', 'map', 'maps', 'steps']

bus book bus journeys biggest bus network country buy bus ticket buy tickets bus journey
drive bus
['course', 'department', 'diagram', 'employees', 'license', 'material', 'position', 'qualifications', 'speed', 'step', 'study']

bus find bus take city want find bus find bus itinerary cities given date
take bus brisbane
[]

bus find bus take city 

/Users/jarana/.local/lib/python3.6/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


[]

event get tickets coolest concerts sports area buy event tickets buy tickets event
get tickets sold event lizardtickets.com
[]

flight find next flight search oneway flight search one-way flights destination
save flight game microsoft flight simulator x
[]

flight find next flight search roundtrip flights search round-trip flights destination
search flights hipmunk
['location', 'text']

flight find next flight reserve oneway flight reserve one-way flight
save flight game microsoft flight simulator x
[]

flight find next flight reserve roundtrip flights reserve round-trip flight
save flight game microsoft flight simulator x
[]

flight search cheap flights across multiple providers search oneway flight search one way flight set preferences
save flight game microsoft flight simulator x
[]

flight search cheap flights across multiple providers search roundtrip flights search round trip flights set preferences
search flights hipmunk
['location', 'text']

home widely used service finding

['event', 'events', 'line', 'option']

event comprehensive portal find reserve seats events near buy event tickets buy tickets event
get tickets sold event lizardtickets.com
[]

flight find one way round trip flights favorite city search oneway flight search one-way flights destination choice
search flights hipmunk
['location', 'text']

flight find one way round trip flights favorite city search roundtrip flights search round-trip flights destination choice
search flights hipmunk
['location', 'text']

home widely used service finding apartments scheduling visits find apartment find apartment city given number bedrooms
make apartment sims 2 apartment life
['area', 'tool']



KeyboardInterrupt: 

In [241]:
props = [camel_case_split(i) for i in orgProp.label.tolist()]

In [15]:
with open("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt") as f:
    for line in f:
        print(line)
        break

{"task_id": "4a06139e", "domain": "UPDATE_CALENDAR", "bot_prompt": "Schedule the user's meeting request", "bot_role": "You are a bot designed to help schedule meetings on a calendar. ", "user_prompt": " You have a meeting saved for March 24th. Ask the chatbot to delete the meeting", "user_role": "You are interacting with a meeting scheduling bot"}



In [13]:
tasks = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt", lines=True)

In [14]:
dialog = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/ORDER_PIZZA.txt", lines=True)

In [23]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/*.txt"):
    txtfiles.append(file)

In [24]:
metalwoz = pd.read_json(txtfiles[0], lines=True)
for txt in txtfiles[1:]:
    tmp = pd.read_json(txt, lines=True)
    metalwoz = pd.concat([metalwoz,tmp])


In [80]:
slots = []
for idx, row in schema.iterrows():
#     print(row)
    service = row['service_name']
    for i in row.slots:
        slots.append(service+"_"+i['name'])
#     for i in row.intents:
#         slots.extend([service+"_"+j for j in i['required_slots']])
#         slots.extend([service+"_"+j for j in list(i['optional_slots'].keys())])
    

In [534]:
import glob

txtfiles = []
dialog = None
for i in ['train', 'dev', 'test']:
    for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/%s/*.json" % i):
        txtfiles.append(file)
        dialog = pd.concat([dialog, pd.read_json(file)])

In [535]:
domains = []
for i in dialog.services.tolist():
    try:
        if len(i) == 1:
            domains.append(i[0])
    except:
        continue

In [539]:
from sklearn.model_selection import train_test_split
trainIds, testIds = train_test_split(list(set(domains)), test_size=0.3, random_state=2020)

In [602]:
print(testIds)

['Calendar_1', 'Music_3', 'Buses_2', 'Music_1', 'Hotels_2', 'Flights_1', 'Flights_4', 'Movies_1', 'Travel_1', 'Banks_1', 'Services_2', 'Services_3', 'RentalCars_1', 'Homes_1']


In [520]:
prop2desc = {}
for idx, s in schema.iterrows():
    service = s.service_name
    for i in s.slots:
        prop2desc[service+"_"+i['name']] = i['description']


In [521]:
len(prop2desc), schema.service_name.nunique()

(365, 45)

In [551]:
tmp = dialog[dialog.services.str.len() == 1]

In [553]:
tmp['domain'] = [i[0] for i in tmp['services']]

/Users/jarana/anaconda3/envs/torch1/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [603]:
tmp.groupby('domain',as_index = False).apply(lambda x: x.sample(20))

dialogue_id     services  \
0  25      500025.0    [Alarm_1]   
   32      500032.0    [Alarm_1]   
   31      500031.0    [Alarm_1]   
   27      500027.0    [Alarm_1]   
   6       300006.0    [Alarm_1]   
...             ...          ...   
43 125     600125.0  [Weather_1]   
   122     600122.0  [Weather_1]   
   0       700000.0  [Weather_1]   
   8       700008.0  [Weather_1]   
   7       700007.0  [Weather_1]   

                                                    turns service_name  \
0  25   [{'frames': [{'service': 'Alarm_1', 'slots': [...          NaN   
   32   [{'frames': [{'service': 'Alarm_1', 'slots': [...          NaN   
   31   [{'frames': [{'service': 'Alarm_1', 'slots': [...          NaN   
   27   [{'frames': [{'service': 'Alarm_1', 'slots': [...          NaN   
   6    [{'frames': [{'service': 'Alarm_1', 'slots': [...          NaN   
...                                                   ...          ...   
43 125  [{'frames': [{'service': 'Weather_1', 'slots':...          NaN   
   122  [{'frames': [{'service': 'Weather_1', 'slots':...          NaN   
   0    [{'frames': [{'service': 'Weather_1', 'slots':...          NaN   
   8    [{'frames': [{'service': 'Weather_1', 'slots':...          NaN   
   7    [{'frames': [{'service': 'Weather_1', 'slots':...          NaN   

       description slots intents     domain  
0  25          NaN   NaN     NaN    Alarm_1  
   32          NaN   NaN     NaN    Alarm_1  
   31          NaN   NaN     NaN    Alarm_1  
   27          NaN   NaN     NaN    Alarm_1  
   6           NaN   NaN     NaN    Alarm_1  
...            ...   ...     ...        ...  
43 125         NaN   NaN     NaN  Weather_1  
   122         NaN   NaN     NaN  Weather_1  
   0           NaN   NaN     NaN  Weather_1  
   8           NaN   NaN     NaN  Weather_1  
   7           NaN   NaN     NaN  Weather_1  

[880 rows x 8 columns]

In [606]:
train_data = []
test_data = []

qid = 1
# for idx, row in dialog.iterrows():
# for idx, row in tmp.groupby('domain',as_index = False).apply(lambda x: x.sample(20)).iterrows():
for idx, row in tmp.iterrows():
    frames = row['turns']
    context = []
    
    if str(frames)=='nan':
        continue
        
    for f in frames:
        service = f['frames'][0]['service']
        if len(f['frames']) == 1:
            utterance = f['utterance']
            if len(f['frames'][0]['slots']) != 0:
                terms = []
                for s in f['frames'][0]['slots']:
                    _context = " ".join(context)
                    answer_start = len(_context) + s["start"] + 1
                    new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
                    _context += " " + new_answer
                    _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
                                                          'question': prop2desc[service+"_"+s['slot']], 'answers':[{'text':s['slot'].replace("_"," "), 
                                                           'answer_start': answer_start}]}]}
                    if service in trainIds:
                        train_data.append(_json)
                    else:
                        test_data.append(_json)
                    qid += 1
            context.append(utterance)


In [607]:
len(train_data), len(test_data)

(40994, 32380)

In [599]:
### Save as a JSON file
import json
with open('data/dialog/train30.json', 'w') as f:
    json.dump(train_data, f)
import json
with open('data/dialog/test30.json', 'w') as f:
    json.dump(test_data, f)